# Custom Soil Property Modifier for WRF Geogrid Files

**Authors:** Juan Sanchez & Matias Suarez  
**Context:** PhD Programme — WRF Custom Preprocessing Tools  
**File:** `mod_soil_properties.ipynb`  

---

## 📌 Description

This tool modifies **soil category variables** in a WRF `geo_em.d0X.nc` geogrid file using a **shapefile** that defines polygon-based regions with custom soil values.

The following variables are updated:
- `SCT_DOM`: Soil Category Top (dominant)
- `SCB_DOM`: Soil Category Bottom (dominant)
- `SOILCTOP`: 16-category boolean vector for top layer
- `SOILCBOT`: 16-category boolean vector for bottom layer

Each polygon in the shapefile assigns specific values to these variables for all enclosed grid cells.

To facilitate usage and testing, this tool includes:
- A **sample `geo_em.d01.nc` file**, and  
- A **sample shapefile** (`test.shp`) with polygon geometries and soil category IDs.  

These example files allow users to run and validate the script directly, and serve as templates.

---

## 📥 Required Inputs

### 1. **Shapefile**
- Format: Polygon (not multipolygon)
- Must include a column `id` that uniquely identifies each polygon
- Used to determine spatial extent of soil modifications

### 2. **WRF Geogrid File**
- Format: NetCDF (`geo_em.d0X.nc`)
- Must contain `XLAT_M`, `XLONG_M`, `SCT_DOM`, `SCB_DOM`, `SOILCTOP`, `SOILCBOT` variables

### 3. **Example Format of values_dict**
Defines the soil categories per polygon ID. Structure:

```python
values_dict = {
    'SCT_DOM': {
        1: 3,         # Polygon with ID 1 → assign soil category 3
        2: 15,        # Polygon with ID 2 → assign soil category 15
        3: 2,         # and so on...
        'def_val': 3  # Default value for grid points outside any polygon
    },
    'SCB_DOM': {
        1: 3,         # Polygon with ID 1 → assign soil category 3
        2: 15,        # Polygon with ID 2 → assign soil category 15
        3: 2,         # and so on...
        'def_val': 3  # Default value for grid points outside any polygon
    }
}
```
---

## 📁 Output

This notebook creates a copy of the original **geo_em.d0X.nc** file named **custom_geo_em.d0X.nc**

The file is saved in the same directory as the original and includes the modified soil variables.

Your original file remains unchanged.

In [1]:
# install netCDF4 library
!pip install netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.2 MB/s eta 0:00:00


In [2]:
# import necessary libraries
import numpy as np
from netCDF4 import Dataset
import geopandas as gpd
from shapely.geometry import Point
import os

In [3]:
# connection to our drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 📂 Required Input: Shapefile

Please provide a **shapefile containing polygon geometries** that define the regions where soil categories should be modified.

In [4]:
# read the shapefile
# only two variables are necessary: id and geometry
shapefile = gpd.read_file('/content/drive/Shareddrives/Aplicaciones WRF - WRF Hydro/10- Codigos-Scripts/custom_soil_properties/shapes/test.shp')
shapefile.set_index('id', inplace=True)
shapefile

,soiltop,dom,geometry
id,,,
1,5.0,1,"POLYGON ((-64.76553 -31.96451, -64.73843 -31.9..."
2,1.0,2,"POLYGON ((-64.6788 -31.98387, -64.64008 -31.94..."
3,2.0,3,"POLYGON ((-64.78644 -32.09964, -64.79961 -32.1..."


### 🛠️ Define the Variables to Be Modified

Each key in the dictionary corresponds to a variable in the WRF geogrid file (e.g., `'SCT_DOM'`, `'SCB_DOM'`).  
The subkeys are the **polygon IDs** defined in the shapefile, and their values represent the **soil category** to assign to grid points located within each polygon.

⚠️ Important Notes
Polygon IDs used in the dictionary must exactly match the 'id' field in the shapefile.
(You can edit the IDs using QGIS or other GIS software if needed.)

Soil category values must be integers from 1 to 16, as required by WRF.

In [11]:
# format example:
# values_dict = {'SCT_DOM': {id polygon 1 : soil category value for points enclosed by polygon 1,
#                            id polygon 2 : soil category value for the points enclosed by polygon 2,
#                            id polygon 3 : soil category value for points enclosed by polygon 3,
#                            and so on...
#                            default value : value of soil category for points not enclosed by any polygon }}

values_dict = {'SCT_DOM': {1 : 6,
                           2 : 15,
                           3 : 7,
                   'def_val' : 3},

               'SCB_DOM': {1 : 6,
                           2 : 15,
                           3 : 7,
                   'def_val' : 3}}

### 📂 Provide the Path to Your `geo_em` File

Please specify the full path to your original WRF `geo_em.d0X.nc` file in the variable `geofile_path`.

This block will automatically:

1. Extract the folder and filename from the provided path.
2. Create a **copy** of the original file, renamed as `custom_<original_filename>.nc`, in the same folder.
3. Open the copied file in read-write mode for modification.

🔒 This approach ensures the **original geogrid file remains unchanged**, allowing you to safely apply changes to the custom copy without risk of overwriting your original data.


In [12]:
# open geogrid file in read-write mode
# It is a good practice to have a copy of the geogrid file
geofile_path = "/content/drive/Shareddrives/Aplicaciones WRF - WRF Hydro/10- Codigos-Scripts/custom_soil_properties/geo_em.d01.nc"

# Get the directory path of the original geogrid file
base_path = os.path.dirname(geofile_path)
# Get the filename from the full path (e.g., 'geo_em.d01.nc')
filename = os.path.basename(geofile_path)
# Create a new path for the copied file by prepending 'custom_' to the original filename
# The result will be something like: '/path/to/folder/custom_geo_em.d01.nc'
geofile_path_cp = base_path+'/custom_'+filename

# Copy the original geogrid file to the new path using a shell command
# This creates a backup named 'custom_<original_filename>' in the same folder
!cp "{geofile_path}" "{geofile_path_cp}"

try:
    ncfile = Dataset(geofile_path_cp, 'r+')
    print(f"✅ Geogrid file loaded successfully: {geofile_path_cp}")
except FileNotFoundError:
    raise FileNotFoundError(f"❌ File not found: {geofile_path_cp}. Please check the path.")

✅ Geogrid file loaded successfully: /content/drive/Shareddrives/Aplicaciones WRF - WRF Hydro/10- Codigos-Scripts/custom_soil_properties/custom_geo_em.d01.nc


### ⚠️ Do Not Modify the Following Code Block

The next code block contains essential logic required for the correct execution of the tool.

Please **do not edit or delete** this section unless you fully understand its functionality.

Any unintended modification may result in incorrect outputs or failure during processing.


In [13]:
# number of rows and columns of the geogrid file
nrows = ncfile['XLONG_M'][0,:,:].shape[0]
ncols = ncfile['XLONG_M'][0,:,:].shape[1]

# set all values of the SOILCBOT and SOILCTOP variables to 0
ncfile['SOILCBOT'][0,:,:,:] = 0
ncfile['SOILCTOP'][0,:,:,:] = 0

# iterate over all dictionary keys
for var in values_dict.keys():

  # set default values outside the polygons
  ncfile[var][0,:,:] = values_dict[var]['def_val']
  # loop through all rows of the geogrid file
  for irow in range(nrows):
    # loop through all columns of the geogrid file
    for icol in range(ncols):
      # get latitude and longitude of each grid point
      lon = float(ncfile['XLONG_M'][0,irow,icol].data)
      lat = float(ncfile['XLAT_M'] [0,irow,icol].data)
      # iterate through all polygons in the shapefile to check if the grid point is inside the polygon or not
      for ipol in shapefile.index:
        if gpd.GeoSeries(shapefile['geometry'][ipol]).contains(Point(lon,lat)).tolist()[0] == True:
          # modify the value of the variable in the geogrid file by the value defined in the dictionary
          ncfile[var][0,irow,icol] = int(values_dict[var][ipol])
          # if-block to modify the soil category variable by setting 0 or 1
          if var == 'SCB_DOM':
            ncfile['SOILCBOT'][0,int(values_dict[var][ipol]-1),irow,icol] = 1
          elif var == 'SCT_DOM':
            ncfile['SOILCTOP'][0,int(values_dict[var][ipol]-1),irow,icol] = 1

In [14]:
# close file
ncfile.close()